In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#install the kaggle
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rushilunagariya","key":"8f9d567c14b247d35624f3493657c154"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                 title                                  size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------  ------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ruchi798/data-science-job-salaries                  Data Science Job Salaries               7KB  2022-06-15 08:59:12          17088        535  1.0              
faryarmemon/usa-housing-market-factors              U.S. Housing Market Factors            32KB  2022-08-03 02:19:31            349         28  1.0              
zzettrkalpakbal/full-filled-brain-stroke-dataset    Brain stroke prediction dataset        52KB  2022-07-16 09:57:08           1842         58  0.9705882        
himanshunakrani/student-study-hours                 Student Study Hours                    276B  2022-07-20 13:17:29           1398         55  1.0              
jillanisofttech/brain-stroke

In [ ]:
!kaggle competitions download -c dogs-vs-cats

100% 809M/812M [00:04<00:00, 177MB/s]
100% 812M/812M [00:04<00:00, 172MB/s]


In [ ]:
!unzip dogs-vs-cats.zip

Archive:  dogs-vs-cats.zip
  inflating: sampleSubmission.csv    
  inflating: test1.zip               
  inflating: train.zip               


In [ ]:
!unzip train.zip

In [ ]:
!unzip test1.zip

# Start of code

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,Activation,BatchNormalization
import os
from sklearn.model_selection import train_test_split
import random
import pathlib
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
Image_Width = 256
Image_Height = 256
Image_Size = (Image_Width,Image_Height)
Image_Channels = 3

In [ ]:
filenames=os.listdir("./train")
categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)
df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

In [ ]:
df

,filename,category
0,cat.10733.jpg,0
1,cat.2947.jpg,0
2,cat.9199.jpg,0
3,dog.9050.jpg,1
4,dog.1977.jpg,1
...,...,...
24995,cat.7921.jpg,0
24996,cat.9671.jpg,0
24997,dog.726.jpg,1
24998,dog.9812.jpg,1


In [ ]:
model=Sequential()

model.add(Conv2D(32,(3,3), activation='relu', input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer ='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 127, 127, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 125, 125, 64)     2

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [ ]:
# managing the categories
df["category"] = df["category"].replace({0:'cat',1:"dog"})

In [ ]:
train_df,validate_df = train_test_split(df,test_size=0.20,random_state=42)#here we divided dataset into two random dataset(train and validate)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=64

In [ ]:
train_df

,filename,category
0,dog.1579.jpg,dog
1,cat.1558.jpg,cat
2,cat.11847.jpg,cat
3,cat.2592.jpg,cat
4,dog.8810.jpg,dog
...,...,...
19995,dog.11319.jpg,dog
19996,dog.781.jpg,dog
19997,dog.7642.jpg,dog
19998,cat.11664.jpg,cat


In [ ]:
validate_df

,filename,category
0,dog.12160.jpg,dog
1,cat.5817.jpg,cat
2,dog.739.jpg,dog
3,cat.2254.jpg,cat
4,cat.10188.jpg,cat
...,...,...
4995,dog.8402.jpg,dog
4996,dog.932.jpg,dog
4997,dog.1145.jpg,dog
4998,dog.7171.jpg,dog


In [ ]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                  rescale=1./255,
                                  zoom_range=0.1,
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1)
train_generator = train_datagen.flow_from_dataframe(train_df,"./train",
                                                    x_col='filename',y_col='category',
                                                    terget_size=Image_Size,
                                                    class_mode='categorical',
                                                    batch_size = batch_size
                                                   )

Found 20000 validated image filenames belonging to 2 classes.


In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df,
    "./train",
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

Found 5000 validated image filenames belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rotation_range=15,
                                 rescale=1./255,
                                 shear_range=0.1,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1)

test_generator = train_datagen.flow_from_directory("/content/test1",
                                                  color_mode = 'rgb',
                                                  target_size=Image_Size,
                                                  class_mode=None,
                                                  batch_size=batch_size,
                                                  shuffle = False)

Found 0 images belonging to 0 classes.


In [ ]:
epochs=10
history = model.fit(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

Epoch 1/10
312/312 [==============================] - 362s 1s/step - loss: 0.8043 - accuracy: 0.6451 - val_loss: 0.6640 - val_accuracy: 0.6056 - lr: 0.0010
Epoch 2/10
312/312 [==============================] - 337s 1s/step - loss: 0.5597 - accuracy: 0.7160 - val_loss: 0.5113 - val_accuracy: 0.7372 - lr: 0.0010
Epoch 3/10
312/312 [==============================] - 331s 1s/step - loss: 0.5038 - accuracy: 0.7515 - val_loss: 0.4970 - val_accuracy: 0.7618 - lr: 0.0010
Epoch 4/10
312/312 [==============================] - 333s 1s/step - loss: 0.4710 - accuracy: 0.7766 - val_loss: 0.4584 - val_accuracy: 0.7833 - lr: 0.0010
Epoch 5/10
312/312 [==============================] - 332s 1s/step - loss: 0.4255 - accuracy: 0.8054 - val_loss: 0.4164 - val_accuracy: 0.8131 - lr: 0.0010
Epoch 6/10
312/312 [==============================] - 331s 1s/step - loss: 0.3952 - accuracy: 0.8217 - val_loss: 0.4940 - val_accuracy: 0.8127 - lr: 0.0010
Epoch 7/10
312/312 [==============================] - 333s 1s/st

In [ ]:
model.save("/content/drive/MyDrive/Data_Science/Cat_vs_Dog/model1_catsVSdogs_10epoch.h5")

In [ ]:
test_filenames = os.listdir("./test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [ ]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


In [ ]:
test_df['category'] = np.argmax(predict, axis=-1)

label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Data_Science/Cat_vs_Dog/model1_catsVSdogs_10epoch.h5') 

In [ ]:
import cv2

In [ ]:
imgs = []
labels = []

for i in os.listdir('./test1'):
  img = cv2.imread('./test1/' + i)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img_rgb, (256, 256))
  img = np.expand_dims(img, axis=0)
  pred = model.predict(img)

  imgs.append(i)
  labels.append(np.argmax(pred))

In [ ]:
print(imgs)
print(labels)

['38.jpg']
[1]


In [ ]:
df = pd.DataFrame()

df['images'] = imgs
df['labels'] = labels

df.to_csv('/content/drive/MyDrive/Data_Science/Cat_vs_Dog/sample.csv')